
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.


In [14]:
import ee
import geemap
import math

# ee.Authenticate()
ee.Initialize(project="ee-zuzannaslobodzian2")


## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [3]:
aoi = ee.Geometry.Point([19.937803, 50.061634])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [89]:
image = ee.ImageCollection("LANDSAT/LC08/C02/T1").filterBounds(aoi).filterDate('2021-08-01', '2021-08-31').sort('CLOUDY_PIXEL_PERCENTAGE').first()
image


## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [90]:
def radiance_calculator(band):
    M_L = ee.Number(image.get(f'RADIANCE_MULT_BAND_{band}'))
    A_L = ee.Number(image.get(f'RADIANCE_ADD_BAND_{band}'))
    DN = image.select(f'B{band}')
    
    print("M_L: ", M_L.getInfo())
    print("A_L: ", A_L.getInfo())

    radiance = DN.multiply(M_L).add(A_L).rename(f'B{band}_rad')
    
    return radiance



## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [91]:
ESUN = {
    'B2': 2067,
    'B3': 1893,
    'B4': 1603,
    'B5': 972
}

sun_elevation = ee.Number(image.getInfo().get('properties', {}).get('SUN_ELEVATION'))
sun_zenith_rad = ee.Number(90).subtract(sun_elevation).multiply(math.pi).divide(180)
cos_theta = sun_zenith_rad.cos()

def toa_reflectance(band):
    L = ee.Image(radiance_calculator(band))
    esun_bn = ee.Number(ESUN[f'B{band}'])
    return L.multiply(math.pi).divide(esun_bn.multiply(cos_theta))

b2 = toa_reflectance(2)
b3 = toa_reflectance(3)
b4 = toa_reflectance(4)
b5 = toa_reflectance(5)

M_L:  0.012526
A_L:  -62.62895
M_L:  0.011542
A_L:  -57.71204
M_L:  0.0097332
A_L:  -48.66605
M_L:  0.0059562
A_L:  -29.78119



## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [95]:
rgb_dn = image.select(['B4', 'B3', 'B2'])

rgb_toa = ee.Image.cat([b2.rename('B2_toa'), b3.rename('B3_toa'), b4.rename('B4_toa'), b5.rename('B5_toa')])
rgb_toa = rgb_toa.clamp(0, 0.4)

Map = geemap.Map()
Map.centerObject(aoi, zoom = 8)
Map.addLayer(rgb_dn, {'min': 0, 'max': 10000}, 'RGB Image')
Map.addLayer(rgb_toa, {'min': 0, 'max': 0.4}, 'TOA Reflectance RGB')
Map

Map(center=[50.061634, 19.937803], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [ ]:
rgb_vis_dn = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 10000,
    'gamma': 1.4
}
rgb_vis_toa = {
    'bands': ['B4_toa', 'B3_toa', 'B2_toa'],
    'min': 0.0,
    'max': 0.4,
    'gamma': 1.2
}

ndvi_vis = {'min': 0, 'max': 1, 'palette': ['white', 'green']}

Map = geemap.Map(center=[50.0647, 19.945], zoom=10)
Map.addLayer(image, rgb_vis_dn, 'Original DN RGB')
Map.addLayer(toa_image.clamp(0, 0.4), rgb_vis_toa, 'TOA Reflectance RGB')
Map.addLayer(ndvi.clip(aoi), ndvi_vis, 'NDVI')
Map.addLayer(aoi, {'color': 'red'}, 'AOI – Kraków')

Map

Map(center=[50.0647, 19.945], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDat…


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
